In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
def save_data_to_csv(input_data, output_path_in):
    input_data.to_csv(output_path_in, index=False)

In [3]:
def filter_rows_based_on_conditions(df, conditions):
    """
    Filter rows in a dataframe based on the given conditions.

    Parameters:
    - df (DataFrame): The input dataframe.
    - conditions (str): A string that specifies the conditions for filtering.

    Returns:
    - DataFrame: A filtered dataframe.
    """
    return df.query(conditions)

In [4]:
def filter_data_on_supplychain_finance(file_path_in):
    # Load the dataset
    data = catalog.load(file_path_in)
    
    # Convert 'Start date' and 'End date' columns to datetime format by inferring the format and coercing errors
    data['Start date'] = pd.to_datetime(data['Start date'], infer_datetime_format=True, errors='coerce')
    data['End date'] = pd.to_datetime(data['End date'], infer_datetime_format=True, errors='coerce')
    #infer_datetime_format is deprecated. Remember to sort this out

    # Filter out rows with NaT values in the "Start date" or "End date" columns
    data = data.dropna(subset=['Start date', 'End date'])

    # Filter the dataset where 'Supply-chain financing offered' is True
    conditions = "`Supply-chain financing offered` == True"
    filtered_data = filter_rows_based_on_conditions(data, conditions)

    # Sort the filtered data by 'Start date'
    sorted_data = filtered_data.sort_values(by='Start date')

    # Save the sorted data to a new CSV file
    save_data_to_csv(sorted_data, output_path)
    return sorted_data 


In [ ]:
def extract_payment_periods(data):
    # Filter data for records starting from 2017 onwards
    periods_from_2017 = data[data['Start date'].dt.year >= 2017]

    # Initialize a dictionary to store periods by year
    payment_periods = defaultdict(list)

    for _, row in periods_from_2017.iterrows():
        # Get the years for start and end dates
        start_year = row['Start date'].year
        end_year = row['End date'].year

        # Convert the start and end dates to "Year Month" format
        start_date_str = row['Start date'].strftime('%Y %b').upper()
        end_date_str = row['End date'].strftime('%Y %b').upper()
        
        # Create the period tuple
        period = f"({start_date_str}, {end_date_str})"
        
        # Add the period to the start year's list
        if period not in payment_periods[start_year]:
            payment_periods[start_year].append(period)
        
        # If the period spans multiple years, add to the end year's list as well
        if start_year != end_year:
            if period not in payment_periods[end_year]:
                payment_periods[end_year].append(period)
    
    return dict(payment_periods)


In [ ]:
def gdpRemoveHeaders():
 
    # Load the data
    data = catalog.load("monthly-gdp-uk.csv")

    # Rename 'Title' column to 'Date'
    data.rename(columns={'Title': 'Date'}, inplace=True)

    # Drop rows before '1997 JAN'
    index_to_drop = data[data['Date'] == '1997 JAN'].index[0]
    data_cleaned = data.loc[index_to_drop:].reset_index(drop=True)

    # Save the cleaned data if necessary
    save_data_to_csv(data_cleaned, "monthly-gdp-uk-headers-removed.csv")

In [ ]:
def calculate_gdp_averages_for_period(start_date, end_date, gdp_file_path="monthly-gdp-uk-headers-removed.csv"):
    # Load the cleaned GDP dataset
    gdp_data = pd.read_csv(gdp_file_path)
    
    # Convert the 'Date' column to datetime format for easy filtering
    gdp_data['Date'] = pd.to_datetime(gdp_data['Date'], format='%Y %b')
    
    # Filter the dataset based on the given period
    period_data = gdp_data[(gdp_data['Date'] >= start_date) & (gdp_data['Date'] <= end_date)]
    
    # Calculate the average for each column over the period
    averages = period_data.mean().drop('Date')  # Drop the 'Date' column as it's not needed in averages
    
    return averages

In [ ]:
def create_period_column(data):
    """Add 'Period' column to the data."""
    data['Period'] = data['Start date'].dt.strftime('%Y %b').str.upper() + " - " + data['End date'].dt.strftime('%Y %b').str.upper()
    return data

In [ ]:
### code related to the inflation rates 
def convert_date_format(df, column_name, format='%d-%b-%y'):
    df[column_name] = pd.to_datetime(df[column_name], format=format)
    return df

In [ ]:
def create_period_column(df, start_col, end_col):
    df["Period"] = df[start_col].dt.strftime('%Y %b').str.upper() + " - " + df[end_col].dt.strftime('%Y %b').str.upper()
    return df

In [ ]:
def calculate_mean_rate_for_periods(payment_df, inflation_df):
    periods = payment_df["Period"].unique()
    mean_rates = {}
    for period in periods:
        start_date, end_date = period.split(" - ")
        start_date = pd.to_datetime(start_date, format='%Y %b')
        end_date = pd.to_datetime(end_date, format='%Y %b')

        mask = (inflation_df["Date Changed"] >= start_date) & (inflation_df["Date Changed"] <= end_date)
        filtered_data = inflation_df[mask]
        
        mean_rate = filtered_data["Rate"].mean()
        
        mean_rates[period] = mean_rate

    payment_df["Mean Inflation Rate"] = payment_df["Period"].map(mean_rates)
    return payment_df

In [ ]:
def combine_datasets(paymentent_practices_path, gdp_averages_path, inflation_rates_path, output_combined_path):
    # Load the datasets
    payment_df = pd.read_csv(payment_practices_path)

    # Explicitly convert 'Start date' and 'End date' columns to datetime format
    payment_df['Start date'] = pd.to_datetime(payment_df['Start date'])
    payment_df['End date'] = pd.to_datetime(payment_df['End date'])

    gdp_df = pd.read_csv(gdp_averages_path)
    inflation_df = pd.read_csv(inflation_rates_path)

    # Convert date format for inflation dataset
    inflation_df = convert_date_format(inflation_df, "Date Changed", format='%d-%b-%y')

    # Create 'Period' column for payment practices dataset
    payment_df = create_period_column(payment_df, "Start date", "End date")

    # Calculate mean inflation rate for each period
    payment_df = calculate_mean_rate_for_periods(payment_df, inflation_df)

    # Merge GDP dataset with payment practices dataset on 'Period' column
    combined_df = pd.merge(gdp_df, payment_df, on="Period", how="left")

    # Save combined dataset
    save_data_to_csv(combined_df, output_combined_path)


In [ ]:
def prepare_inflation_data(inflation_data_path, start_date='2017-01-01'):
    """
    Generate a monthly inflation rate dataset from the start_date onwards,
    forward fill the missing rates and calculate averages for specified periods.
    """
    # Load the inflation data
    inflation_df = pd.read_csv(inflation_data_path)
    
    # Convert date format for inflation dataset
    inflation_df = convert_date_format(inflation_df, "Date Changed", format='%d-%b-%y')
    
    # Set the Date as the index
    inflation_df.set_index('Date Changed', inplace=True)
    
    # Create a monthly date range starting from the specified date
    date_rng = pd.date_range(start=start_date, end=pd.Timestamp.today(), freq='MS')
    
    # Create a new DataFrame with the date range and forward fill the rates
    monthly_inflation_df = pd.DataFrame(date_rng, columns=['Date']).set_index('Date')
    monthly_inflation_df = monthly_inflation_df.join(inflation_df).ffill()
    
    return monthly_inflation_df.reset_index()

In [ ]:
def get_average_inflation_for_periods(inflation_df, periods):
    """
    Calculate the average inflation rate for a list of periods.
    """
    mean_rates = {}
    for period in periods:
        start_date, end_date = period.split(" - ")
        start_date = pd.Timestamp(start_date) # Do i need timestamp??
        end_date = pd.Timestamp(end_date)
    
        ## Must verify this logic and ensure that it is actually getting means for correct periods
        ## to me looks like its not actually comparing the period but instead date col only has one date
        ## possibly just the start date.
        ## Need to print or output the inflation df being used to verify.
        ## Print mean rates dict too to see what it looks like.
        mask = (inflation_df["Date"] >= start_date) & (inflation_df["Date"] <= end_date)
        filtered_data = inflation_df[mask]
        
        mean_rate = filtered_data["Rate"].mean()
        
        mean_rates[period] = mean_rate

    return mean_rates


In [ ]:
def forward_fill_missing_inflation_rates(df, column_name="Mean Inflation Rate"):
    """
    Forward fills missing values in the specified column.
    """
    df[column_name].fillna(method='ffill', inplace=True)
    return df

In [ ]:
def main():
    input_file_path = "payment-practices.csv"  
    output_file_path = "filtered_payment_practices.csv"  
    averages_csv = "gdp_averages.csv"
    inflation_rates_path = "bank-of-england_inflation-rates-uk.csv"  
    output_combined_path = "final_combined_dataset.csv"

    filtered_and_sorted_payment_practise_data = filter_data_on_supplychain_finance(input_file_path, output_file_path)

    payment_periods = extract_payment_periods(filtered_and_sorted_payment_practise_data )



    # Generate a monthly inflation dataset
    monthly_inflation = prepare_inflation_data(inflation_rates_path)

    # Extract periods from the payment practices dataset
    # Extract periods from the payment practices dataset
    payment_practises_df = pd.read_csv(output_file_path)

    # Convert 'Start date' and 'End date' columns to datetime format
    payment_practises_df['Start date'] = pd.to_datetime(payment_practises_df['Start date'])
    payment_practises_df['End date'] = pd.to_datetime(payment_practises_df['End date'])

    payment_practises_df = create_period_column(payment_practises_df, "Start date", "End date")
    periods = payment_practises_df["Period"].unique()

    
    payment_practises_with_period_column_df = create_period_column(payment_practises_df, "Start date", "End date")
    periods = payment_practises_with_period_column_df["Period"].unique() # Store unique payment practise periods 

    # Calculate average inflation rates for the periods
    avg_inflation_rates = get_average_inflation_for_periods(monthly_inflation, periods)
    payment_practises_with_period_column_df["Mean Inflation Rate"] = payment_practises_with_period_column_df["Period"].map(avg_inflation_rates)
    save_data_to_csv(payment_practises_with_period_column_df, output_file_path)# Overwrite the original file with the new column added

    gdpRemoveHeaders()
    all_averages = []
    all_periods = []  # To capture the 'Period' for each average

    for year_periods in payment_periods.values():
        for period in year_periods:
            start, end = period[1:-1].split(', ')  # Split the period string into start and end dates
            all_periods.append(f"{start} - {end}")
            averages_for_period = calculate_gdp_averages_for_period(start, end)
            all_averages.append(averages_for_period)
    
    # This is the part that is resulting in the period column and the gdp data being on the left most 
    # side of the data instead of the payments practises being first. 
    # look into how to switch these around. 
    # Convert all averages to a DataFrame
    averages_df = pd.DataFrame(all_averages)
    averages_df.insert(0, 'Period', all_periods)  # Inserting the 'Period' column at the beginning

    # Save the averages to a CSV
#     averages_df.to_csv(averages_csv, index=False)
    save_data_to_csv( averages_df, averages_csv)

    # Combine the datasets
    combine_datasets(output_file_path, averages_csv, inflation_rates_path, output_combined_path)
    
    # This doesnt seem like it should belong here. Idealy should be done at the stage that the 
    # inflatio rates file is completed, instead of completing it then forward filling at this stage.
    # .. or initially it should check if there are missing values and if there is, they get forward filled.
    # Forward fill missing inflation rates
    combined_df = pd.read_csv(output_combined_path)
    save_data_to_csv(forward_fill_missing_inflation_rates(combined_df), output_combined_path)

    # Print messages
    print(f"Filtered and sorted data saved to {output_file_path}")
    print(f"GDP averages saved to {averages_csv}")
    print(f"Combined dataset saved to {output_combined_path}")
